In [3]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [14]:
label_name = 'Hráč vyhrál'

In [15]:
df = pd.read_csv(r"./data/1_st_benesov_raw.csv")
df

,Hráč,Oddíl,Družstvo,Soupeř,Oddíl soupeř,Družstvo soupeř,Výsledek,Datum zápasu,Soutěž,Elo hráče,...,Elo (max),Elo soupeře,Elo (min) soupeře,Elo (max) soupeře,Rok narození,Max elo,Elo nejlepšího poraženého soupeře,ID Hráč,ID Soupeř,Rok narození soupeř
0,Daniel Miffek,Stolní tenis Benešov,ST Benešov A,Pavel Zatřepálek,"TTC Brandýs n.L, spolek",TTC Brandýs n.L C,3:0,27.11.2021,Krajská soutěž 1.třídy B,1929-> 1931,...,1981-> 1982,1770-> 1769,1753-> 1752,1785-> 1784,1976,1977,2118,miffek_daniel_1976,zatrepalek_pavel_1968,1968
1,Daniel Miffek,Stolní tenis Benešov,ST Benešov A,Luboš Mühlfeit,"TTC Brandýs n.L, spolek",TTC Brandýs n.L C,3:0,27.11.2021,Krajská soutěž 1.třídy B,1929-> 1929,...,1981-> 1981,1646-> 1646,1637-> 1637,1661-> 1661,1976,1977,2118,miffek_daniel_1976,muhlfeit_lubos_1975,1975
2,Daniel Miffek,Stolní tenis Benešov,ST Benešov A,Milan Ivan,"TTC Brandýs n.L, spolek",TTC Brandýs n.L C,3:0,27.11.2021,Krajská soutěž 1.třídy B,1929-> 1929,...,1981-> 1981,1523-> 1523,1511-> 1511,1533-> 1533,1976,1977,2118,miffek_daniel_1976,ivan_milan_1964,1964
3,Daniel Miffek,Stolní tenis Benešov,ST Benešov A,Petr Zatřepálek,"TTC Brandýs n.L, spolek",TTC Brandýs n.L C,3:0,27.11.2021,Krajská soutěž 1.třídy B,1928-> 1929,...,1980-> 1981,1728-> 1728,1711-> 1711,1746-> 1745,1976,1977,2118,miffek_daniel_1976,zatrepalek_petr_1995,1995
4,Daniel Miffek,Stolní tenis Benešov,ST Benešov A,Dalibor Šulc,Tělocvičná jednota Sokol Libčice nad Vltavou,Sokol Libčice nad Vltavou A,3:1,30.10.2021,Krajská soutěž 1.třídy B,1928-> 1928,...,1978-> 1978,1540-> 1540,1504-> 1504,1569-> 1569,1976,1977,2118,miffek_daniel_1976,sulc_dalibor_1953,1953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8232,Miroslav Mrázek,Stolní tenis Benešov,Stolní tenis Benešov E,Petr Vodička,Sokol Divišov,Sokol Divišov C,2:3,12.10.2018,Okresní přebor 4,944-> 943,...,966-> 964,1109-> 1110,1086-> 1088,1131-> 1132,1964,1022,1263,miroslav_mrazek_1964,petr_vodicka_1971,1971
8233,Miroslav Mrázek,Stolní tenis Benešov,Stolní tenis Benešov E,Jaroslav Sládek ml.,TJ Sokol Teplýšovice,TJ Sokol Teplýšovice B,3:0,5.10.2018,Okresní přebor 4,944-> 944,...,966-> 966,1-> 1,1-> 1,1077-> 1029,1964,1022,1263,miroslav_mrazek_1964,jaroslav_sladek_2000,2000
8234,Miroslav Mrázek,Stolní tenis Benešov,Stolní tenis Benešov E,Dominik Grézl,TJ Sokol Teplýšovice,TJ Sokol Teplýšovice B,2:3,5.10.2018,Okresní přebor 4,950-> 944,...,974-> 966,909-> 917,885-> 894,933-> 940,1964,1022,1263,miroslav_mrazek_1964,dominik_grezl_1975,1975
8235,Miroslav Mrázek,Stolní tenis Benešov,Stolní tenis Benešov E,Vladimír Škvor,TJ Sokol Teplýšovice,TJ Sokol Teplýšovice B,3:0,5.10.2018,Okresní přebor 4,950-> 950,...,974-> 974,1-> 1,1-> 1,1104-> 1042,1964,1022,1263,miroslav_mrazek_1964,vladimir_skvor_1967,1967


In [16]:
from datetime import datetime as dt

def get_season(dt):
    if dt.month >= 3 and dt.month <= 5:
        return 1
    if dt.month >= 6 and dt.month <= 8:
        return 2
    if dt.month >= 9 and dt.month <= 11:
        return 3
    return 4

def petisetovka(hrac, souper):
    hrac = int(hrac)
    souper = int(souper)
    if (hrac < 2) or (souper < 2): return None
    if hrac > souper: return True
    return False

In [53]:
df['Hráč sety'] = df['Výsledek'].str[0]
df['Soupeř sety'] = df['Výsledek'].str[2]
df['Hráč vyhrál'] = df['Hráč sety'] > df['Soupeř sety']
df['Datum zápasu'] = pd.to_datetime(df['Datum zápasu'])
df['Den v týdnu (číslo)'] = df['Datum zápasu'].dt.weekday + 1
df['Elo hráče před'] = df['Elo hráče'].str.split('->', expand=True)[0]
df['Elo hráče po'] = df['Elo hráče'].str.split('->', expand=True)[1]
df['Elo soupeře před'] = df['Elo soupeře'].str.split('->', expand=True)[0]
df['Soupeř je žena'] = df['Soupeř'].apply(lambda x: False if re.search(".*ová$", x) is None else True)
df['Hráč je žena'] = df['Hráč'].apply(lambda x: False if re.search(".*ová$", x) is None else True)
df['Datum zápasu'] = pd.to_datetime(df['Datum zápasu'])
df['isWeekend'] = df['Den v týdnu (číslo)'] >= 6
df['obdobi'] = df['Datum zápasu'].apply(lambda x: get_season(x))
X = df[['Rok narození', 'Elo hráče před', 'Hráč je žena',
        'Rok narození soupeř', 'Elo soupeře před', 'Soupeř je žena',
        'Den v týdnu (číslo)', 'isWeekend', 'obdobi']]
y = df[label_name]

X

,Rok narození,Elo hráče před,Hráč je žena,Rok narození soupeř,Elo soupeře před,Soupeř je žena,Den v týdnu (číslo),isWeekend,obdobi
0,1976,1929,False,1968,1770,False,6,True,3
1,1976,1929,False,1975,1646,False,6,True,3
2,1976,1929,False,1964,1523,False,6,True,3
3,1976,1928,False,1995,1728,False,6,True,3
4,1976,1928,False,1953,1540,False,6,True,3
...,...,...,...,...,...,...,...,...,...
8232,1964,944,False,1971,1109,False,1,False,4
8233,1964,944,False,2000,1,False,4,False,1
8234,1964,950,False,1975,909,False,4,False,1
8235,1964,950,False,1967,1,False,4,False,1


In [54]:
ss = StandardScaler()
scaled = ss.fit_transform(X)
scaled_df = pd.DataFrame(scaled)
scaled_df.columns = X.columns
scaled_df

array([[-6.72357426e-04,  2.48657274e+00,  0.00000000e+00, ...,
         5.97240556e-01,  1.12711885e+00,  8.63033980e-02],
       [-6.72357426e-04,  2.48657274e+00,  0.00000000e+00, ...,
         5.97240556e-01,  1.12711885e+00,  8.63033980e-02],
       [-6.72357426e-04,  2.48657274e+00,  0.00000000e+00, ...,
         5.97240556e-01,  1.12711885e+00,  8.63033980e-02],
       ...,
       [-6.39696371e-01, -2.52890377e+00,  0.00000000e+00, ...,
        -5.27852745e-01, -8.87217879e-01, -1.84806691e+00],
       [-6.39696371e-01, -2.52890377e+00,  0.00000000e+00, ...,
        -5.27852745e-01, -8.87217879e-01, -1.84806691e+00],
       [-6.39696371e-01, -2.53914989e+00,  0.00000000e+00, ...,
        -5.27852745e-01, -8.87217879e-01, -1.84806691e+00]])

In [58]:
scaled_df.to_csv('data/3_X.csv', index=False)
y.to_csv('data/3_y.csv', index=False)